In [1]:
import sys
import mosek
import numpy as np

In [2]:
import random
#Generate problem data
np.random.seed(10)
m = 10 #Number of stores
n = 5 #Number of warehouses
r = [5,2,9,3,5,10,11,6,9,4] #Required number of widgets
s = [15,17,18,15,8] #Widget supply
#Shipping costs
C = np.round(10*np.random.rand(m,n))+1 #Shipping costs


In [3]:
#Convert C to list of lists
L = []
for i in range(0,10):
    L.append(C[i,:].tolist())

In [4]:
#Dictionary to store problem data
shipping_data = {'store_requirements': r, 'widget_supply': s, 'shipping_costs': L}

In [5]:
import json
with open("data.json", "w") as outfile: 
    json.dump(shipping_data, outfile)

In [6]:
A = np.vstack((np.kron(np.eye(m),np.ones(n).T),
               np.kron(-np.ones(m).T,np.eye(n)))) # Constraint matrix
r = np.array(r)
s = np.array(s)

In [7]:
# Since the value of infinity is ignored, we define it solely
# for symbolic purposes
inf = 0.0

# Define a stream printer to grab output from MOSEK
def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

In [8]:
def main():
    # Create a task object
    with mosek.Task() as task:
        # Attach a log stream printer to the task
        task.set_Stream(mosek.streamtype.log, streamprinter)

        # Bound keys for constraints
        #bkc = [mosek.boundkey.fx,
        #       mosek.boundkey.lo,
        #       mosek.boundkey.up]
        
        bkc = [mosek.boundkey.lo]*(m+n)

        # Bound values for constraints
        #blc = [30.0, 15.0, -inf]
        blc = np.hstack((r,-s))
        buc = +inf*np.ones(m+n) 

        # Bound keys for variables
        #bkx = [mosek.boundkey.lo,
        #       mosek.boundkey.ra,
        #       mosek.boundkey.lo,
        #       mosek.boundkey.lo]
        
        bkx = [mosek.boundkey.lo]*(m*n)

        # Bound values for variables
        #blx = [0.0, 0.0, 0.0, 0.0]
        blx = np.zeros(m*n)
        bux = +inf*np.ones(m*n)
        #bux = [+inf, 10.0, +inf, +inf]

        # Objective coefficients
        c = np.ravel(C)

        # Below is the sparse representation of the A
        # matrix stored by column.
        asub = [[0, 1],
                [0, 1, 2],
                [0, 1],
                [1, 2]]
        aval = [[3.0, 2.0],
                [1.0, 1.0, 2.0],
                [2.0, 3.0],
                [1.0, 3.0]]

        numvar = len(bkx)
        numcon = len(bkc)

        # Append 'numcon' empty constraints.
        # The constraints will initially have no bounds.
        task.appendcons(numcon)

        # Append 'numvar' variables.
        # The variables will initially be fixed at zero (x=0).
        task.appendvars(numvar)

        for j in range(numvar):
            # Set the linear term c_j in the objective.
            task.putcj(j, c[j])

            # Set the bounds on variable j
            # blx[j] <= x_j <= bux[j]
            task.putvarbound(j, bkx[j], blx[j], bux[j])

            # Input column j of A
            J = np.nonzero(A[:,j])[0]
            aval = A[:,j][J]
            task.putacol(j,                  # Variable (column) index.
                         J,            # Row index of non-zeros in column j.
                         aval)            # Non-zero Values of column j.

        # Set the bounds on constraints.
         # blc[i] <= constraint_i <= buc[i]
        for i in range(numcon):
            task.putconbound(i, bkc[i], blc[i], buc[i])

        # Input the objective sense (minimize/maximize)
        task.putobjsense(mosek.objsense.minimize)
        
         # Define variables to be integers
        task.putvartypelist(np.arange(0,m*n),[mosek.variabletype.type_int]*(m*n))

        # Set max solution time
        task.putdouparam(mosek.dparam.mio_max_time, 60.0)

        # Optimize the task
        task.optimize()
        task.writedata("milo1.ptf")

        # Print a summary containing information
        # about the solution for debugging purposes
        task.solutionsummary(mosek.streamtype.msg)

        prosta = task.getprosta(mosek.soltype.itg)
        solsta = task.getsolsta(mosek.soltype.itg)

        # Output a solution         
        xx = task.getxx(mosek.soltype.itg)

        if solsta in [mosek.solsta.integer_optimal]:
            print("Optimal solution: %s" % xx)
        elif solsta == mosek.solsta.prim_feas:
            print("Feasible solution: %s" % xx)
        elif mosek.solsta.unknown:
            if prosta == mosek.prosta.prim_infeas_or_unbounded:
                print("Problem status Infeasible or unbounded.\n")
            elif prosta == mosek.prosta.prim_infeas:
                print("Problem status Infeasible.\n")
            elif prosta == mosek.prosta.unkown:
                print("Problem status unkown.\n")
            else:
                print("Other problem status.\n")
        else:
            print("Other solution status")
        return xx

In [12]:
try:
    xx = main()
except mosek.Error as e:
    print("ERROR: %s" % str(e.errno))
    if e.msg is not None:
        print("\t%s" % e.msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 15              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 50              
  Matrix variables       : 0               
  Integer variables      : 50              

Optimizer started.
Mixed integer optimizer started.
Threads used: 8
Presolve started.
Presolve terminated. Time = 0.00, probing time =  0.00
Presolved problem: 50 variables, 15 constraints, 100 non-zeros
Presolved problem: 50 general integer, 0 binary, 0 continuous
Clique table size: 0
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        1        0        1.7800000000e+02     1.7800000000e+02     0.00e+00    0.0   
An optimal solution satisfying the relative gap tolera

In [3]:
C

array([[ 9.,  1.,  7.,  8.,  6.],
       [ 3.,  3.,  9.,  3.,  2.],
       [ 8., 11.,  1.,  6.,  9.],
       [ 7.,  8.,  4., 10.,  8.],
       [ 6.,  2.,  5.,  8.,  5.],
       [ 5.,  7.,  6.,  8.,  7.],
       [ 9.,  6., 10.,  4.,  2.],
       [ 4.,  2.,  9.,  1.,  7.],
       [ 6.,  9.,  3., 10.,  5.],
       [ 9.,  4., 10.,  4.,  3.]])

In [4]:
x = np.split(C,5)

In [25]:
L = []
for i in range(0,10):
    L.append(x[i].tolist())

In [26]:
L[9]

[[9.0, 4.0, 10.0, 4.0, 3.0]]

In [13]:
L[0]

[[9.0, 1.0, 7.0, 8.0, 6.0], [3.0, 3.0, 9.0, 3.0, 2.0]]

In [14]:
x

[array([[9., 1., 7., 8., 6.],
        [3., 3., 9., 3., 2.]]),
 array([[ 8., 11.,  1.,  6.,  9.],
        [ 7.,  8.,  4., 10.,  8.]]),
 array([[6., 2., 5., 8., 5.],
        [5., 7., 6., 8., 7.]]),
 array([[ 9.,  6., 10.,  4.,  2.],
        [ 4.,  2.,  9.,  1.,  7.]]),
 array([[ 6.,  9.,  3., 10.,  5.],
        [ 9.,  4., 10.,  4.,  3.]])]

In [15]:
C[0,:]

array([9., 1., 7., 8., 6.])

In [18]:
x = np.split(C,10)

In [21]:
x

[array([[9., 1., 7., 8., 6.]]),
 array([[3., 3., 9., 3., 2.]]),
 array([[ 8., 11.,  1.,  6.,  9.]]),
 array([[ 7.,  8.,  4., 10.,  8.]]),
 array([[6., 2., 5., 8., 5.]]),
 array([[5., 7., 6., 8., 7.]]),
 array([[ 9.,  6., 10.,  4.,  2.]]),
 array([[4., 2., 9., 1., 7.]]),
 array([[ 6.,  9.,  3., 10.,  5.]]),
 array([[ 9.,  4., 10.,  4.,  3.]])]

In [27]:
C.shape

(10, 5)

In [13]:
A

array([[ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,
         1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
     